In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.externals import joblib

%matplotlib inline

### Import model and files

In [104]:
## define what's needed for model to load

from sklearn.base import BaseEstimator, TransformerMixin

class FeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column    
    def fit(self, X, y=None):
        return self 
    def transform(self, X, y=None):
        return X[self.column].values

In [105]:
# load up the 
filename = 'logreg_ppg.pkl'
pipeline = pickle.load(open(filename, 'rb'))

In [106]:
## import files needed
model_df = pd.read_csv('./data/model_df.csv')
pred_full = pd.read_csv('./data/pred_full.csv')
pred_full.head()

,target,year-player,player,award_tm,listed_pos,age,g,gs,mp,per,...,orb_per_game,drb_per_game,trb_per_game,ast_per_game,stl_per_game,blk_per_game,tov_per_game,pf_per_game,pts_per_game,probas
0,0,1989.0kareem abdul-jabbar,kareem abdul-jabbar,0,C,41.0,74.0,74.0,1695.0,12.9,...,1.391892,3.121622,4.513514,1.000000,0.513514,1.148649,1.283784,2.648649,10.108108,0.004268
1,0,1989.0mark acres,mark acres,0,C,26.0,62.0,0.0,632.0,8.2,...,0.951613,1.403226,2.354839,0.306452,0.306452,0.096774,0.370968,1.516129,2.209677,0.000021
2,0,1989.0michael adams,michael adams,0,PG,26.0,77.0,77.0,2787.0,17.5,...,0.922078,2.753247,3.675325,6.363636,2.155844,0.142857,2.337662,1.935065,18.493506,0.010129
3,0,1989.0mark aguirre,mark aguirre,0,SF,29.0,80.0,76.0,2597.0,16.4,...,1.825000,3.000000,4.825000,3.475000,0.562500,0.450000,2.600000,2.862500,18.887500,0.003385
4,0,1989.0danny ainge,danny ainge,0,PG,29.0,73.0,54.0,2377.0,18.0,...,0.972603,2.520548,3.493151,5.506849,1.273973,0.109589,1.986301,2.547945,17.547945,0.004648


In [107]:
pred_full['year'] = pred_full['year-player'].apply(lambda x: float(x[:4]))

### Need to re-do the predicted dataframe for the whole dataset now
- Have the workflow from previous notebook

In [108]:
pred_names = pd.DataFrame()

for year in pred_full['year'].unique():
    for pos in model_df.award_pos.unique():
        
        maskyear = year
        maskpos = pos

        mask = (pred_full['pos_encode_'+maskpos] == 1) & (pred_full['year'] == maskyear)
        
        
        ## control flow:
        if maskpos != 'C':
        
            pred_names = pred_names.append(pred_full.loc[mask, ['year', 'player', 'probas', 'target']].sort_values(by='probas', ascending=False).head(6))
                    
        else:
            pred_names = pred_names.append(pred_full.loc[mask, ['year', 'player', 'probas', 'target']].sort_values(by='probas', ascending=False).head(3))

        
pred_names = pred_names.reset_index(drop=True)

## comment
pred_award_pos = [1, 2, 3, 1, 1, 2, 2, 3, 3, 1, 1, 2, 2, 3, 3] * pred_names.year.nunique()
pred_names['pred_award_tm'] = pred_award_pos


## comment
pred_names['year-player'] = pred_names['year'].astype(str) + pred_names['player']
pred_names['predicted'] = 1

pred_names = pd.merge(pred_names, model_df[['year-player', 'award_pos']], how='left', on='year-player')

pred_names.head(15)

,year,player,probas,target,pred_award_tm,year-player,predicted,award_pos
0,1989.0,hakeem olajuwon,0.993966,1,1,1989.0hakeem olajuwon,1,C
1,1989.0,patrick ewing,0.741323,1,2,1989.0patrick ewing,1,C
2,1989.0,moses malone,0.719781,0,3,1989.0moses malone,1,C
3,1989.0,michael jordan,0.999682,1,1,1989.0michael jordan,1,G
4,1989.0,magic johnson,0.989883,1,1,1989.0magic johnson,1,G
5,1989.0,john stockton,0.981688,1,2,1989.0john stockton,1,G
6,1989.0,dale ellis,0.933171,1,2,1989.0dale ellis,1,G
7,1989.0,clyde drexler,0.886493,0,3,1989.0clyde drexler,1,G
8,1989.0,kevin johnson,0.630598,1,3,1989.0kevin johnson,1,G
9,1989.0,karl malone,0.997444,1,1,1989.0karl malone,1,F


In [109]:
## save it!
pred_names.to_csv('./data/pred_names_FINAL.csv', index=False)

Yippee!! Now we have a full prediction set. Time to re-append to `pred_full` to do analysis, and also to keep!

In [110]:
pred_full[(pred_full['year'] == 2000) & (pred_full['target'] > 0)].head()

,target,year-player,player,award_tm,listed_pos,age,g,gs,mp,per,...,drb_per_game,trb_per_game,ast_per_game,stl_per_game,blk_per_game,tov_per_game,pf_per_game,pts_per_game,probas,year
2858,1,2000.0kobe bryant,kobe bryant,2,SG,21.0,66.0,62.0,2524.0,21.7,...,4.666667,6.303030,4.893939,1.606061,0.939394,2.757576,3.333333,22.500000,0.913072,2000.0
2872,1,2000.0vince carter,vince carter,3,SF,23.0,82.0,82.0,3126.0,23.4,...,3.975610,5.804878,3.926829,1.341463,1.121951,2.170732,3.207317,25.695122,0.870080,2000.0
2919,1,2000.0tim duncan,tim duncan,1,PF,23.0,74.0,74.0,2875.0,24.8,...,8.864865,12.405405,3.162162,0.891892,2.229730,3.270270,2.837838,23.189189,0.997186,2000.0
2942,1,2000.0kevin garnett,kevin garnett,1,PF,23.0,81.0,81.0,3243.0,23.6,...,9.049383,11.802469,4.950617,1.481481,1.555556,3.308642,2.530864,22.925926,0.922176,2000.0
2977,1,2000.0grant hill,grant hill,2,SF,27.0,74.0,74.0,2776.0,24.5,...,5.310811,6.621622,5.202703,1.391892,0.581081,3.243243,2.567568,25.756757,0.958184,2000.0


In [111]:
## merge it!
pred_full_merge = pd.merge(pred_full, pred_names[['year-player', 'award_pos', 'predicted', 'pred_award_tm']], 
         how='left', on='year-player').fillna(0)

In [112]:
pred_full_merge[(pred_full_merge['year'] == 2000) & (pred_full_merge['target'] > 0)].head()

,target,year-player,player,award_tm,listed_pos,age,g,gs,mp,per,...,stl_per_game,blk_per_game,tov_per_game,pf_per_game,pts_per_game,probas,year,award_pos,predicted,pred_award_tm
2858,1,2000.0kobe bryant,kobe bryant,2,SG,21.0,66.0,62.0,2524.0,21.7,...,1.606061,0.939394,2.757576,3.333333,22.500000,0.913072,2000.0,G,1.0,1.0
2872,1,2000.0vince carter,vince carter,3,SF,23.0,82.0,82.0,3126.0,23.4,...,1.341463,1.121951,2.170732,3.207317,25.695122,0.870080,2000.0,F,1.0,3.0
2919,1,2000.0tim duncan,tim duncan,1,PF,23.0,74.0,74.0,2875.0,24.8,...,0.891892,2.229730,3.270270,2.837838,23.189189,0.997186,2000.0,F,1.0,1.0
2942,1,2000.0kevin garnett,kevin garnett,1,PF,23.0,81.0,81.0,3243.0,23.6,...,1.481481,1.555556,3.308642,2.530864,22.925926,0.922176,2000.0,F,1.0,3.0
2977,1,2000.0grant hill,grant hill,2,SF,27.0,74.0,74.0,2776.0,24.5,...,1.391892,0.581081,3.243243,2.567568,25.756757,0.958184,2000.0,F,1.0,2.0


In [113]:
## save it!
# pred_full_merge.to_csv('./data/pred_full_merge.csv', index=False)

## bop it
merged_full = pd.read_csv('./data/pred_full_merge.csv')

### Onto our analysis!

For some reason, the year 2000 is missing its `target` and `award_tm` data...

** Biggest Snubs**  
Needs a "so who got it instead?"

In [120]:
mask = (pred_full_merge['target'] == 0) & (pred_full_merge['predicted'] == 1)

pred_full_merge.loc[mask].sort_values(by='probas', ascending=False).head(8)

,target,year-player,player,award_tm,listed_pos,age,g,gs,mp,per,...,stl_per_game,blk_per_game,tov_per_game,pf_per_game,pts_per_game,probas,year,award_pos,predicted,pred_award_tm
11658,0,1993.0shaquille o'neal,shaquille o'neal,0,C,20.0,81.0,81.0,3071.0,22.9,...,0.740741,3.530864,3.790123,3.962963,23.370370,0.994671,1993.0,C,1.0,2.0
8436,0,2016.0james harden,james harden,0,SG,26.0,82.0,82.0,3125.0,25.3,...,1.695122,0.621951,4.560976,2.792683,28.975610,0.966763,2016.0,G,1.0,2.0
9926,0,2001.0jerry stackhouse,jerry stackhouse,0,SG,26.0,80.0,80.0,3215.0,21.8,...,1.212500,0.675000,4.075000,2.000000,29.750000,0.944586,2001.0,G,1.0,2.0
5669,0,2008.0allen iverson,allen iverson,0,SG,32.0,82.0,82.0,3424.0,20.9,...,1.951220,0.146341,2.987805,1.329268,26.390244,0.936964,2008.0,G,1.0,2.0
4692,0,2006.0kevin garnett,kevin garnett,0,PF,29.0,76.0,76.0,2957.0,26.8,...,1.368421,1.407895,2.368421,2.710526,21.789474,0.931423,2006.0,F,1.0,3.0
78,0,1989.0clyde drexler,clyde drexler,0,SG,26.0,78.0,78.0,3064.0,23.6,...,2.730769,0.692308,3.205128,3.448718,27.217949,0.886493,1989.0,G,1.0,3.0
5508,0,2008.0chauncey billups,chauncey billups,0,PG,31.0,78.0,78.0,2522.0,23.6,...,1.294872,0.217949,2.051282,1.666667,16.974359,0.866341,2008.0,G,1.0,3.0
7181,0,2012.0derrick rose,derrick rose,0,PG,23.0,39.0,39.0,1375.0,23.0,...,0.897436,0.717949,3.051282,1.333333,21.846154,0.824353,2012.0,G,1.0,2.0


** Biggest "Cupcakes" **  
Needs a "so who should've gotten it?"

In [121]:
mask = (pred_full_merge['target'] == 1) & (pred_full_merge['predicted'] == 0)

pred_full_merge.loc[mask].sort_values(by='probas', ascending=True).head(8)

,target,year-player,player,award_tm,listed_pos,age,g,gs,mp,per,...,stl_per_game,blk_per_game,tov_per_game,pf_per_game,pts_per_game,probas,year,award_pos,predicted,pred_award_tm
11813,1,2009.0carmelo anthony,carmelo anthony,3,SF,24.0,66.0,66.0,2277.0,19.0,...,1.136364,0.363636,3.015152,2.954545,22.787879,0.007532,2009.0,0,0.0,0.0
11678,1,1993.0drazen petrovic,drazen petrovic,3,SG,28.0,70.0,67.0,2660.0,17.3,...,1.342857,0.185714,2.914286,3.385714,22.342857,0.008645,1993.0,0,0.0,0.0
1826,1,1995.0dennis rodman,dennis rodman,3,PF,33.0,49.0,26.0,1568.0,16.6,...,0.632653,0.469388,2.000000,3.244898,7.122449,0.009262,1995.0,0,0.0,0.0
6895,1,2012.0tyson chandler,tyson chandler,3,C,29.0,62.0,62.0,2061.0,18.7,...,0.903226,1.435484,1.645161,3.000000,11.274194,0.016848,2012.0,0,0.0,0.0
2670,1,1998.0scottie pippen,scottie pippen,3,SF,32.0,44.0,44.0,1652.0,20.4,...,1.795455,0.977273,2.477273,2.636364,19.113636,0.017044,1998.0,0,0.0,0.0
9294,1,1996.0juwan howard,juwan howard,3,PF,22.0,81.0,81.0,3294.0,17.0,...,0.827160,0.481481,3.740741,3.320988,22.086420,0.023642,1996.0,0,0.0,0.0
3048,1,2000.0stephon marbury,stephon marbury,3,PG,22.0,74.0,74.0,2881.0,20.7,...,1.513514,0.202703,3.648649,2.635135,22.162162,0.032085,2000.0,0,0.0,0.0
9832,1,2001.0dikembe mutombo,dikembe mutombo,2,C,34.0,75.0,75.0,2591.0,17.5,...,0.386667,2.706667,1.920000,2.720000,9.986667,0.049195,2001.0,0,0.0,0.0


**Biggest Bridesmaid**  
Needs a "is there a year he should've made it?"

In [136]:
mask = (pred_full_merge['target'] == 0)

pred_full_merge.loc[mask, ['player', 'year', 'pts_per_game', 'probas', 'predicted','pred_award_tm']]\
.sort_values(by='probas', ascending=False).head(100).groupby('player').agg("count")\
.sort_values(by='year', ascending=False).head(25)

,year,pts_per_game,probas,predicted,pred_award_tm
player,,,,,
vince carter,3,3,3,3,3
patrick ewing,3,3,3,3,3
carmelo anthony,3,3,3,3,3
shawn marion,3,3,3,3,3
reggie miller,2,2,2,2,2
damian lillard,2,2,2,2,2
allen iverson,2,2,2,2,2
larry nance,2,2,2,2,2
isiah thomas,2,2,2,2,2


In [142]:
mask = (pred_full_merge['target'] == 0)

pred_full_merge.loc[mask, ['player', 'year', 'pts_per_game', 'probas', 'predicted','pred_award_tm']]\
.sort_values(by='probas', ascending=False).head(100).groupby('player').agg("sum")\
.sort_values(by='probas', ascending=False).head(25)

,year,pts_per_game,probas,predicted,pred_award_tm
player,,,,,
patrick ewing,5985.0,70.930879,2.217277,0.0,0.0
vince carter,6018.0,73.927282,2.069940,2.0,6.0
carmelo anthony,6033.0,78.701299,1.675254,3.0,9.0
shawn marion,6011.0,56.051799,1.468598,1.0,3.0
alonzo mourning,3991.0,44.523377,1.328720,0.0,0.0
tony parker,4013.0,37.433442,1.311486,0.0,0.0
allen iverson,4012.0,52.765244,1.237465,2.0,5.0
brad daugherty,3982.0,39.079270,1.098749,0.0,0.0
damian lillard,4032.0,47.962276,1.043686,1.0,3.0


In [139]:
## checking out Melo
mask = (pred_full_merge['target'] == 0) & (pred_full_merge['player'] == 'carmelo anthony')

pred_full_merge.loc[mask].sort_values(by='probas', ascending=False).head(3)

,target,year-player,player,award_tm,listed_pos,age,g,gs,mp,per,...,stl_per_game,blk_per_game,tov_per_game,pf_per_game,pts_per_game,probas,year,award_pos,predicted,pred_award_tm
7305,0,2014.0carmelo anthony,carmelo anthony,0,PF,29.0,77.0,77.0,2982.0,24.4,...,1.233766,0.662338,2.571429,2.909091,27.428571,0.815980,2014.0,F,1.0,3.0
5481,0,2008.0carmelo anthony,carmelo anthony,0,SF,23.0,77.0,77.0,2806.0,21.1,...,1.272727,0.506494,3.285714,3.285714,25.688312,0.434489,2008.0,F,1.0,3.0
6374,0,2011.0carmelo anthony,carmelo anthony,0,SF,26.0,77.0,77.0,2751.0,21.7,...,0.883117,0.597403,2.675325,2.909091,25.584416,0.424784,2011.0,F,1.0,3.0


In [144]:
## checking out Patrick Ewing
mask = (pred_full_merge['target'] == 0) & (pred_full_merge['player'] == 'patrick ewing')

pred_full_merge.loc[mask].sort_values(by='probas', ascending=False).head(3)

,target,year-player,player,award_tm,listed_pos,age,g,gs,mp,per,...,stl_per_game,blk_per_game,tov_per_game,pf_per_game,pts_per_game,probas,year,award_pos,predicted,pred_award_tm
1235,0,1994.0patrick ewing,patrick ewing,0,C,31.0,79.0,79.0,2972.0,22.9,...,1.139241,2.746835,3.291139,3.481013,24.544304,0.991980,1994.0,0,0.0,0.0
1633,0,1995.0patrick ewing,patrick ewing,0,C,32.0,79.0,79.0,2920.0,21.9,...,0.860759,2.012658,3.240506,3.443038,23.873418,0.805291,1995.0,0,0.0,0.0
9243,0,1996.0patrick ewing,patrick ewing,0,C,33.0,76.0,76.0,2783.0,20.9,...,0.894737,2.421053,2.907895,3.250000,22.513158,0.420006,1996.0,0,0.0,0.0


In [145]:
## checking out Vince Carter
mask = (pred_full_merge['target'] == 0) & (pred_full_merge['player'] == 'vince carter')

pred_full_merge.loc[mask].sort_values(by='probas', ascending=False).head(3)

,target,year-player,player,award_tm,listed_pos,age,g,gs,mp,per,...,stl_per_game,blk_per_game,tov_per_game,pf_per_game,pts_per_game,probas,year,award_pos,predicted,pred_award_tm
4626,0,2006.0vince carter,vince carter,0,SG,29.0,79.0,79.0,2906.0,21.6,...,1.189873,0.670886,2.696203,2.974684,24.189873,0.796590,2006.0,0,0.0,0.0
5085,0,2007.0vince carter,vince carter,0,SG,30.0,82.0,82.0,3126.0,21.8,...,1.000000,0.365854,2.646341,3.243902,25.243902,0.674559,2007.0,G,1.0,3.0
10065,0,2005.0vince carter,vince carter,0,SG,28.0,77.0,76.0,2828.0,22.9,...,1.415584,0.623377,2.181818,3.155844,24.493506,0.598791,2005.0,G,1.0,3.0


### Let's look at Patrick Ewing
- 3 years he just missed... 1994-1995-1996
- Why?

In [151]:
## checking out Patrick Ewing
mask = (pred_full_merge['year'] == 1994) & (pred_full_merge['listed_pos'] == 'C')

pred_full_merge.loc[mask].sort_values(by='probas', ascending=False).head(6)

,target,year-player,player,award_tm,listed_pos,age,g,gs,mp,per,...,stl_per_game,blk_per_game,tov_per_game,pf_per_game,pts_per_game,probas,year,award_pos,predicted,pred_award_tm
1386,1,1994.0shaquille o'neal,shaquille o'neal,3,C,21.0,81.0,81.0,3224.0,28.5,...,0.938272,2.851852,2.740741,3.469136,29.345679,0.999990,1994.0,C,1.0,1.0
1427,1,1994.0david robinson,david robinson,2,C,28.0,80.0,80.0,3241.0,30.7,...,1.737500,3.312500,3.162500,2.850000,29.787500,0.999984,1994.0,C,1.0,2.0
1389,1,1994.0hakeem olajuwon,hakeem olajuwon,1,C,31.0,80.0,80.0,3277.0,25.3,...,1.600000,3.712500,3.387500,3.612500,27.300000,0.999393,1994.0,C,1.0,3.0
1235,0,1994.0patrick ewing,patrick ewing,0,C,31.0,79.0,79.0,2972.0,22.9,...,1.139241,2.746835,3.291139,3.481013,24.544304,0.991980,1994.0,0,0.0,0.0
1484,0,1994.0chris webber,chris webber,0,C,20.0,76.0,76.0,2438.0,21.7,...,1.223684,2.157895,2.710526,3.250000,17.539474,0.286361,1994.0,0,0.0,0.0
1373,0,1994.0alonzo mourning,alonzo mourning,0,C,23.0,60.0,59.0,2018.0,21.7,...,0.450000,3.133333,3.316667,3.450000,21.450000,0.147987,1994.0,0,0.0,0.0


In [152]:
## checking out Patrick Ewing
mask = (pred_full_merge['year'] == 1995) & (pred_full_merge['listed_pos'] == 'C')

pred_full_merge.loc[mask].sort_values(by='probas', ascending=False).head(6)

,target,year-player,player,award_tm,listed_pos,age,g,gs,mp,per,...,stl_per_game,blk_per_game,tov_per_game,pf_per_game,pts_per_game,probas,year,award_pos,predicted,pred_award_tm
1781,1,1995.0shaquille o'neal,shaquille o'neal,2,C,22.0,79.0,79.0,2923.0,28.6,...,0.924051,2.430380,2.582278,3.265823,29.303797,0.999968,1995.0,C,1.0,1.0
1823,1,1995.0david robinson,david robinson,1,C,29.0,81.0,81.0,3074.0,29.1,...,1.654321,3.234568,2.876543,2.839506,27.629630,0.999592,1995.0,C,1.0,2.0
1783,1,1995.0hakeem olajuwon,hakeem olajuwon,3,C,32.0,72.0,72.0,2853.0,26.0,...,1.847222,3.361111,3.291667,3.472222,27.847222,0.986909,1995.0,C,1.0,3.0
1633,0,1995.0patrick ewing,patrick ewing,0,C,32.0,79.0,79.0,2920.0,21.9,...,0.860759,2.012658,3.240506,3.443038,23.873418,0.805291,1995.0,0,0.0,0.0
1769,0,1995.0alonzo mourning,alonzo mourning,0,C,24.0,77.0,77.0,2941.0,20.1,...,0.636364,2.922078,3.129870,3.571429,21.337662,0.589745,1995.0,0,0.0,0.0
1857,0,1995.0rik smits,rik smits,0,C,28.0,78.0,78.0,2381.0,19.5,...,0.512821,1.012821,2.423077,3.564103,17.948718,0.045790,1995.0,0,0.0,0.0


In [153]:
## checking out Patrick Ewing
mask = (pred_full_merge['year'] == 1996) & (pred_full_merge['listed_pos'] == 'C')

pred_full_merge.loc[mask].sort_values(by='probas', ascending=False).head(6)

,target,year-player,player,award_tm,listed_pos,age,g,gs,mp,per,...,stl_per_game,blk_per_game,tov_per_game,pf_per_game,pts_per_game,probas,year,award_pos,predicted,pred_award_tm
9444,1,1996.0david robinson,david robinson,1,C,30.0,82.0,82.0,3019.0,29.4,...,1.353659,3.304878,2.317073,3.195122,25.012195,0.999752,1996.0,C,1.0,1.0
9394,1,1996.0shaquille o'neal,shaquille o'neal,3,C,23.0,54.0,52.0,1946.0,26.4,...,0.629630,2.129630,2.870370,3.574074,26.555556,0.991646,1996.0,C,1.0,2.0
9397,1,1996.0hakeem olajuwon,hakeem olajuwon,2,C,33.0,72.0,72.0,2797.0,25.5,...,1.569444,2.875000,3.430556,3.361111,26.888889,0.976827,1996.0,C,1.0,3.0
9381,0,1996.0alonzo mourning,alonzo mourning,0,C,25.0,70.0,70.0,2671.0,22.7,...,1.000000,2.700000,3.742857,3.500000,23.185714,0.738974,1996.0,0,0.0,0.0
9243,0,1996.0patrick ewing,patrick ewing,0,C,33.0,76.0,76.0,2783.0,20.9,...,0.894737,2.421053,2.907895,3.250000,22.513158,0.420006,1996.0,0,0.0,0.0
9459,0,1996.0arvydas sabonis,arvydas sabonis,0,C,31.0,73.0,21.0,1735.0,24.7,...,0.876712,1.068493,2.109589,2.890411,14.493151,0.038028,1996.0,0,0.0,0.0
